In [8]:
from tensorflow.keras import layers
from tensorflow.keras.applications import resnet50, ResNet50
from tensorflow.keras.utils import plot_model, image_dataset_from_directory
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow import convert_to_tensor
import tensorflow.keras.backend as K

import tensorflow as tf

In [9]:
import numpy as np
import pathlib
import os

### Loading data with Tensorflow API

In [10]:
input_shape = (224, 224, 3)

In [11]:
left_dir = pathlib.Path('../../../Dataset/img_celeba_pairs/left')
left_image_count = len(list(left_dir.glob('*.jpg')))
print('===left===')
print(left_image_count)

right_dir = pathlib.Path('../../../Dataset/img_celeba_pairs/right')
right_image_count = len(list(right_dir.glob('*.jpg')))
print('===right===')
print(right_image_count)

===left===
30
===right===
30


In [6]:
labels = np.zeros(left_image_count)
labels[1::2] = 1

labels = convert_to_tensor(labels)

Metal device set to: Apple M1 Pro


2022-11-24 14:08:07.610199: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-24 14:08:07.610525: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [12]:
AUTOTUNE = tf.data.AUTOTUNE

left = image_dataset_from_directory(
  left_dir,
  labels=None,
  shuffle=False,
  image_size= input_shape[:2],
  batch_size=32)

right = image_dataset_from_directory(
  right_dir,
  labels=None,
  shuffle=False,
  image_size= input_shape[:2],
  batch_size=32)

def preprocessing(image):
    image = resnet50.preprocess_input(image)
    return image / 255

left = left.map(preprocessing, num_parallel_calls=tf.data.AUTOTUNE)
right = right.map(preprocessing, num_parallel_calls=tf.data.AUTOTUNE)



left = left.cache().prefetch(buffer_size=AUTOTUNE)
right = right.cache().prefetch(buffer_size=AUTOTUNE)

Found 30 files belonging to 1 classes.
Found 30 files belonging to 1 classes.


### Input layers

In [264]:
input_shape = (224, 224, 3)

left_input = layers.Input(shape=target_shape, name='left_input', dtype=tf.float32)
right_input = layers.Input(shape=target_shape, name='right_input', dtype=tf.float32)

### Resnet model

In [265]:
resnet=ResNet50(
    include_top=False,
    input_shape=input_shape,
    pooling='avg',
    weights='imagenet',
    classes=2)

for layer in resnet.layers[0:-1]:
    layer.trainable = False
    
left_resnet = resnet(left_input)
right_resnet = resnet(right_input)

### Merging resnet branches layers

In [266]:
def euclidean_distance(vects):
    x, y = vects
    sum_square = tf.reduce_sum(tf.square(subtract(x, y)), axis=1, keepdims=True)
    return tf.sqrt(tf.maximum(sum_square, K.epsilon()))

In [267]:
#merged = layers.subtract([left_resnet, right_resnet])
merged = layers.Lambda(euclidean_distance, name="merge")([left_resnet, right_resnet])

flatten = layers.Flatten()(merged)

merged_dense_1 = layers.Dense(256, activation='relu')(flatten)
merged_dense_2 = layers.Dense(32, activation='relu')(merged_dense_1)

merged_output = layers.Dense(1, activation='sigmoid')(merged_dense_2)

model = Model(inputs = [left_input, right_input], outputs = merged_output)

In [268]:
# margin is a parametr settable by developer
def contrastive_loss_with_margin(margin):
    def contrastive_loss(y_true, y_pred):
        '''Contrastive loss from Hadsell-et-al.'06
        http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
        '''
        square_pred = tf.square(y_pred)
        margin_square = tf.square(tf.maximum(margin - y_pred, 0))
        return (y_true * square_pred + (1 - y_true) * margin_square)    
    return contrastive_loss

In [269]:
lr=0.001

model.compile(optimizer=Adam(learning_rate=lr), loss=contrastive_loss_with_margin(margin=1.0), metrics=['accuracy'])

In [270]:
#plot_model(resnet, show_shapes=True)
#model.summary()

In [286]:
combo_model_fitted = model.fit(x = [left, right], y=labels, epochs=5)

ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {"<class 'tensorflow.python.data.ops.dataset_ops.PrefetchDataset'>"}), <class 'tensorflow.python.framework.ops.EagerTensor'>

In [ ]:
plt.plot(combo_model_fitted.history['accuracy'])
plt.plot(combo_model_fitted.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.axis(ymin=0.5,ymax=1)
plt.grid()

plt.show()

In [ ]:
plt.plot(combo_model_fitted.history['loss'])
plt.plot(combo_model_fitted.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.axis(ymin=0,ymax=0.5)
plt.grid()

plt.show()

### Predictions

In [ ]:
left_image = X_test[:,0]
left_image_feature_vectors = base_network.predict(left_image)